In [1]:
# HIDDEN - sets path

import sys
sys.path.insert(1, '../..')

# Working with times

## Client
It is important to set the timezone when creating the client. All output times will be given in this timezone, and timezone-unaware inputs are assumed to be in this timezone.

In [2]:
import keyring
from trendminer_interface import TrendMinerClient

url = "https://cs.trendminer.net"
client_id = "tmInterfaceClient"

client = TrendMinerClient(
    url = url,
    client_id = client_id,
    client_secret=keyring.get_password(url, client_id),
    tz="Europe/Brussels"
)

## Timestamps
All single timestamps are returned as `datetime.datetime` instances. Timestamp string inputs will be converted automaticallly. For string inputs, the date portion is assumed to be in YYYY-MM-DD.

We can always get the current time:

In [3]:
client.now()

datetime.datetime(2023, 5, 8, 13, 34, 0, 754536, tzinfo=<DstTzInfo 'Europe/Brussels' CEST+2:00:00 DST>)

Converting a string into a datetime is easy using the built-in datetime method. Note that if the timezone is not given in the string, it is assumed to be in the client timezone.

In [4]:
client.time.datetime("2021-01-12 14:05:00.000")

datetime.datetime(2021, 1, 12, 14, 5, tzinfo=<DstTzInfo 'Europe/Brussels' CET+1:00:00 STD>)

If a timezone is included in the string, the timestamp is converted to the client timezone.

In [5]:
client.time.datetime("2021-01-12 14:05:00.000+09:00")

datetime.datetime(2021, 1, 12, 6, 5, tzinfo=<DstTzInfo 'Europe/Brussels' CET+1:00:00 STD>)

## Durations
All durations are returned as `datetime.timedelta`. Input strings and numeric values are coverted automatically.

Strings are interpreted like they are in the TrendMiner UX

In [6]:
client.time.timedelta("4d 12h 5m 3s")

datetime.timedelta(days=4, seconds=43503)

Numeric inputs are assumed to represent the number of seconds

In [7]:
client.time.timedelta(1800)

datetime.timedelta(seconds=1800)

## Intervals
Intervals are described by a fixed start and end time. They are a core element in the package, and in TrendMiner in general. TrendHub views, context items, search periods, ... they are all defined by an interval.

Intervals can be made from two timestamps (datetimes or strings)

In [8]:
client.interval("2021-01-01", "2022-01-01")

<< Interval | 2021-01-01 00:00:00+01:00 | 365 days, 0:00:00 >>

For as single input, the end is the current time

In [9]:
client.interval("2023-01-10 12:00")

<< Interval | 2023-01-10 12:00:00+01:00 | 118 days, 0:34:00.973638 >>

Duration-like string inputs als take the current time as the end

In [10]:
client.interval("5d 12h")

<< Interval | 2023-05-03 01:34:00.988529+02:00 | 5 days, 12:00:00 >>

### Properties and methods
Intervals have a set of useful properties and methods implemented

Basic properties are start and end times, and duration

In [11]:
interval = client.interval("9h")

print(f"Start: {interval.start}")
print(f"End: {interval.end}")
print(f"Duration: {interval.duration}")

Start: 2023-05-08 04:34:01.001657+02:00
End: 2023-05-08 13:34:01.001657+02:00
Duration: 9:00:00


We can check if one intervals falls completely within the other

In [12]:
interval in client.interval("2022", "2025")

True

We can check how much one interval (nominator) covers the other (denominator)

In [13]:
client.interval("2021", "2023") / client.interval("2022", "2024")

0.5

Intervals can easily be shifted

In [14]:
interval = interval.shift("-1h")

print(f"Start: {interval.start}")
print(f"End: {interval.end}")
print(f"Duration: {interval.duration}")

Start: 2023-05-08 03:34:01.001657+02:00
End: 2023-05-08 12:34:01.001657+02:00
Duration: 9:00:00


Intervals can easily be rounded down to a given resolution

In [15]:
interval = interval.floor(resolution="1h")
                           
print(f"Start: {interval.start}")
print(f"End: {interval.end}")
print(f"Duration: {interval.duration}")

Start: 2023-05-08 04:00:00+02:00
End: 2023-05-08 12:00:00+02:00
Duration: 8:00:00


We can also take the time just before or after a given interval.

In [16]:
print(interval.before("1h"))
print(interval.after("1h"))

<< Interval | 2023-05-08 03:00:00+02:00 | 1:00:00 >>
<< Interval | 2023-05-08 12:00:00+02:00 | 1:00:00 >>


An interval can be split in equal parts based on a *maximal* duration.

In [17]:
interval.split("2.5h")

[<< Interval | 2023-05-08 04:00:00+02:00 | 2:00:00 >>,
 << Interval | 2023-05-08 06:00:00+02:00 | 2:00:00 >>,
 << Interval | 2023-05-08 08:00:00+02:00 | 2:00:00 >>,
 << Interval | 2023-05-08 10:00:00+02:00 | 2:00:00 >>]

We can take random samples of a given length from a longer interval, or from a list of intervals. This is a quick way to obtain sufficient samples for a statistical test.

In [18]:
samples = interval.sample(
    duration="15m", # resulting interval duration
    n=4, # number of intervals
    overlap=False, # whether the intervals are allowed to overlap
    resolution="1m", # rounded to given resolution (index resolution by default)
)

samples

[<< Interval | 2023-05-08 05:20:00+02:00 | 0:15:00 >>,
 << Interval | 2023-05-08 06:06:00+02:00 | 0:15:00 >>,
 << Interval | 2023-05-08 09:25:00+02:00 | 0:15:00 >>,
 << Interval | 2023-05-08 09:56:00+02:00 | 0:15:00 >>]

In [19]:
subsamples = client.interval.sample(
    intervals=samples,
    duration="3m",
    n=12,
    overlap=False,
    resolution="1m"
)

In some cases, we want to inverting intervals, taking the negative space (i.e., the room inbetween the given intervals) within in a larger span

In [20]:
client.interval.invert(samples, span=interval)

[<< Interval | 2023-05-08 04:00:00+02:00 | 1:20:00 >>,
 << Interval | 2023-05-08 05:35:00+02:00 | 0:31:00 >>,
 << Interval | 2023-05-08 06:21:00+02:00 | 3:04:00 >>,
 << Interval | 2023-05-08 09:40:00+02:00 | 0:16:00 >>,
 << Interval | 2023-05-08 10:11:00+02:00 | 1:49:00 >>]

## Periods
Periods are basically live intervals representing the last x time. Their start and end times are constantly moving, which makes them hard to work with. Typically, you want to be working with intervals, not periods. 

Periods are occasionally returned from the appliance when something is represented by a live, moving timeframe.

In [21]:
# A period can be defined from a duration
period = client.time.period("1d")
period

<< Period | 1 day, 0:00:00 >>

In [22]:
# A period is always moving
print(period.start)
print(period.start)

2023-05-07 13:34:01.206666+02:00
2023-05-07 13:34:01.207485+02:00


In [23]:
# We can freeze a period into a fixed interval
period.freeze()

<< Interval | 2023-05-07 13:34:01.214625+02:00 | 1 day, 0:00:00 >>

In [24]:
# Period methods will implicitly freeze the period
period.split("8h")

[<< Interval | 2023-05-07 13:34:01.224355+02:00 | 8:00:00 >>,
 << Interval | 2023-05-07 21:34:01.224355+02:00 | 8:00:00 >>,
 << Interval | 2023-05-08 05:34:01.224355+02:00 | 8:00:00 >>]